In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-02 15:15:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.70MB/s    in 0.2s    

2022-04-02 15:15:07 (5.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://data-finalproject.s3.amazonaws.com/ID_rawdata.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)

In [5]:
df.show(5)


+--------+---------------+---------+-----+--------------------+-----------------------+-----------------+-----------------+--------------+---+----------+---------------+-------+------+------------+------+-----+---------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+----+---------+
|      ID|      timestamp|  company|level|               title|totalyearlycompensation|         location|yearsofexperience|yearsatcompany|tag|basesalary|stockgrantvalue|  bonus|gender|otherdetails|cityid|dmaid|rowNumber|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool|Some_College|Race_Asian|Race_White|Race_Two_Or_More|Race_Black|Race_Hispanic|Race|Education|
+--------+---------------+---------+-----+--------------------+-----------------------+-----------------+-----------------+--------------+---+----------+---------------+-------+------+------------+------+-----+---------+--------------+----------------+--

In [6]:
df.columns

['ID',
 'timestamp',
 'company',
 'level',
 'title',
 'totalyearlycompensation',
 'location',
 'yearsofexperience',
 'yearsatcompany',
 'tag',
 'basesalary',
 'stockgrantvalue',
 'bonus',
 'gender',
 'otherdetails',
 'cityid',
 'dmaid',
 'rowNumber',
 'Masters_Degree',
 'Bachelors_Degree',
 'Doctorate_Degree',
 'Highschool',
 'Some_College',
 'Race_Asian',
 'Race_White',
 'Race_Two_Or_More',
 'Race_Black',
 'Race_Hispanic',
 'Race',
 'Education']

In [5]:
new_df = df.withColumnRenamed("ID", "employee_no")

In [6]:
new_df.show(5)

+-----------+---------------+---------+-----+--------------------+-----------------------+-----------------+-----------------+--------------+---+----------+---------------+-------+------+------------+------+-----+---------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+----+---------+
|employee_no|      timestamp|  company|level|               title|totalyearlycompensation|         location|yearsofexperience|yearsatcompany|tag|basesalary|stockgrantvalue|  bonus|gender|otherdetails|cityid|dmaid|rowNumber|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool|Some_College|Race_Asian|Race_White|Race_Two_Or_More|Race_Black|Race_Hispanic|Race|Education|
+-----------+---------------+---------+-----+--------------------+-----------------------+-----------------+-----------------+--------------+---+----------+---------------+-------+------+------------+------+-----+---------+--------------+----------

In [8]:
salary_info_df = new_df.select(['employee_no','title', 'totalyearlycompensation','yearsofexperience','yearsatcompany','basesalary','stockgrantvalue','bonus']).drop_duplicates()
salary_info_df.show(5)

+-----------+-----------------+-----------------------+-----------------+--------------+----------+---------------+-------+
|employee_no|            title|totalyearlycompensation|yearsofexperience|yearsatcompany|basesalary|stockgrantvalue|  bonus|
+-----------+-----------------+-----------------------+-----------------+--------------+----------+---------------+-------+
|   FLP00014|Software Engineer|                 191000|              7.0|           7.0|    152000|        17000.0|22000.0|
|   FLP00050|  Product Manager|                 170000|              9.0|           2.0|    123000|        37000.0| 1000.0|
|   FLP00189|Software Engineer|                 517000|             11.0|           3.0|    207000|       260000.0|41000.0|
|   FLP00743|Software Engineer|                 358000|             16.0|           3.0|    200000|       128000.0|30000.0|
|   FLP01470|Software Engineer|                 113000|              1.0|           1.0|    108000|         7000.0| 5000.0|
+-------

In [9]:
Company_info_df = new_df.select(['employee_no','company', 'location','cityid']).drop_duplicates()
Company_info_df.show(5)

+-----------+----------+-----------------+------+
|employee_no|   company|         location|cityid|
+-----------+----------+-----------------+------+
|   FLP00256|    Google|Mountain View, CA|  7322|
|   FLP00496|  Facebook|      Seattle, WA| 11527|
|   FLP00590|Salesforce|San Francisco, CA|  7419|
|   FLP01020|     Intel|  Santa Clara, CA|  7434|
|   FLP01608|      eBay|     San Jose, CA|  7422|
+-----------+----------+-----------------+------+
only showing top 5 rows



In [10]:
Employee_info_df = new_df.select(['employee_no', 'yearsofexperience','yearsatcompany','gender','Masters_Degree','Bachelors_Degree','Doctorate_Degree','Highschool']).drop_duplicates()
Employee_info_df.show(5)

+-----------+-----------------+--------------+------+--------------+----------------+----------------+----------+
|employee_no|yearsofexperience|yearsatcompany|gender|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool|
+-----------+-----------------+--------------+------+--------------+----------------+----------------+----------+
|   FLP00127|              7.0|           7.0|    NA|             0|               0|               0|         0|
|   FLP00315|             13.0|           5.5|  Male|             0|               0|               0|         0|
|   FLP00523|              2.0|           2.0|    NA|             0|               0|               0|         0|
|   FLP00550|              3.0|           3.0|  Male|             0|               0|               0|         0|
|   FLP00974|             12.0|          10.0|    NA|             0|               0|               0|         0|
+-----------+-----------------+--------------+------+--------------+----------------+---

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://final-project.cgernf5ijfvg.us-east-1.rds.amazonaws.com/postgres"
config = {"user":"postgres", 
          "password": "finalproject1", 
          "driver":"org.postgresql.Driver"}

In [12]:
salary_info_df.write.jdbc(url=jdbc_url, table="Salary_info", mode=mode, properties=config)
salary_info_df.show()

+-----------+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+
|employee_no|               title|totalyearlycompensation|yearsofexperience|yearsatcompany|basesalary|stockgrantvalue|  bonus|
+-----------+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+
|   FLP00014|   Software Engineer|                 191000|              7.0|           7.0|    152000|        17000.0|22000.0|
|   FLP00050|     Product Manager|                 170000|              9.0|           2.0|    123000|        37000.0| 1000.0|
|   FLP00189|   Software Engineer|                 517000|             11.0|           3.0|    207000|       260000.0|41000.0|
|   FLP00743|   Software Engineer|                 358000|             16.0|           3.0|    200000|       128000.0|30000.0|
|   FLP01470|   Software Engineer|                 113000|              1.0|           1.0|    108000|         

In [13]:
Company_info_df.write.jdbc(url=jdbc_url, table="Company_info", mode=mode, properties=config)
Company_info_df

DataFrame[employee_no: string, company: string, location: string, cityid: string]

In [14]:
Employee_info_df.write.jdbc(url=jdbc_url, table="Employee_info", mode=mode, properties=config)
Employee_info_df

DataFrame[employee_no: string, yearsofexperience: double, yearsatcompany: double, gender: string, Masters_Degree: string, Bachelors_Degree: string, Doctorate_Degree: string, Highschool: string]

In [15]:

salary_info_df = salary_info_df.alias('salary_info_df')
Employee_info_df = Employee_info_df.alias('Employee_info_df')
Company_info_df = Company_info_df.alias('Company_info_df')


In [17]:
from pyspark.sql.functions import col

Joined_df = salary_info_df.join(Employee_info_df, col('salary_info_df.employee_no') == col('Employee_info_df.employee_no'), 'inner') \
.join(Company_info_df, col('salary_info_df.employee_no') == col('Company_info_df.employee_no'), 'inner') 

In [18]:
Cleaned_data_df = Joined_df.drop("employee_no")

In [19]:
Cleaned_data_df.show(5)

+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+-----------------+--------------+------+--------------+----------------+----------------+----------+--------+-----------------+------+
|               title|totalyearlycompensation|yearsofexperience|yearsatcompany|basesalary|stockgrantvalue|  bonus|yearsofexperience|yearsatcompany|gender|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool| company|         location|cityid|
+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+-----------------+--------------+------+--------------+----------------+----------------+----------+--------+-----------------+------+
|Software Engineer...|                 320000|              7.0|           7.0|    160000|       160000.0|    0.0|              7.0|           7.0|    NA|             0|               0|               0|         0|  Amazon|      Seattle, WA| 11527|
|   